In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 19144191
paper_name = 'kemmer_roberge_2009' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_csv('raw_data/hits_genenames.txt', header=None)

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 53 x 1


In [7]:
original_data.head()

,0
0,CUP5
1,STV1
2,TFP1
3,TFP3
4,VMA2


In [8]:
original_data['gene'] = original_data[0].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['gene'] = clean_orf(original_data['gene'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['gene'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [0, gene, orf]
Index: []


In [12]:
original_data['data'] = -1

In [13]:
original_data.set_index('orf', inplace=True)

In [14]:
original_data = original_data[['data']].copy()

In [15]:
original_data = original_data.groupby(original_data.index).mean()

In [16]:
original_data.shape

(53, 1)

# Load & process tested strains

In [19]:
tested = pd.read_excel('raw_data/haploid set.xlsx', sheet_name='haploid set', skiprows=3)

In [20]:
tested.head()

,record no.,ORF name,strain,batch,row/col,re-array,re-array.1,re-array.2,RG notes,QC Notes
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10338.0,YAL068C,BY4739,01_1,A2,101.0,A,2,NaN,NaN
2,10339.0,YAL067C,BY4739,01_1,A3,101.0,A,3,NaN,NaN
3,10340.0,YAL066W,BY4739,01_1,A4,101.0,A,4,NaN,NaN
4,10341.0,YAL065C,BY4739,01_1,A5,101.0,A,5,NaN,NaN


In [21]:
tested['orf'] = tested['ORF name'].astype(str)

In [22]:
tested['orf'] = clean_orf(tested['orf'])

In [23]:
tested.loc[tested['orf']=='YYKL138C','orf'] = 'YKL138C'

In [24]:
tested['orf'] = translate_sc(tested['orf'], to='orf')

In [25]:
# Make sure everything translated ok
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,])

             record no. ORF name strain batch row/col  re-array re-array.1  \
index_input                                                                  
0                   NaN      NaN    NaN   NaN     NaN       NaN        NaN   
4667                NaN      NaN    NaN   NaN     NaN       NaN        NaN   
4828                NaN      NaN    NaN   NaN     NaN       NaN        NaN   

            re-array.2 RG notes QC Notes  orf  
index_input                                    
0                  NaN      NaN      NaN  NAN  
4667           end 150      NaN      NaN  NAN  
4828           end 171      NaN      NaN  NAN  


In [26]:
tested = tested.loc[t,:]

In [27]:
tested_orfs = tested['orf'].unique()

In [28]:
missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
missing

[]

In [29]:
original_data = original_data.reindex(index=tested_orfs, fill_value=0)

# Prepare the final dataset

In [30]:
data = original_data.copy()

In [31]:
dataset_ids = [159]
datasets = datasets.reindex(index=dataset_ids)

In [32]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [33]:
data.head()

dataset_id,159
data_type,value
orf,
YAL068C,0
YAL067C,0
YAL066W,0
YAL065C,0
YAL061W,0


## Subset to the genes currently in SGD

In [34]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 27


In [35]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,159
,data_type,value
gene_id,orf,
1869,YAL068C,0
61,YAL067C,0
60,YAL066W,0
1727,YAL065C,0
56,YAL061W,0


# Normalize

In [36]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [37]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [38]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id        159       
data_type       value valuez
gene_id orf                 
1869    YAL068C     0    0.0
61      YAL067C     0    0.0
60      YAL066W     0    0.0
1727    YAL065C     0    0.0
56      YAL061W     0    0.0

# Print out

In [39]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [40]:
from IO.save_data_to_db3 import *

In [41]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 19144191...
Inserting the new data...


100%|██████████| 1/1 [00:10<00:00, 10.65s/it]

Updating the data_modified_on field...
